In [1]:
import numpy
from openfermion.utils._rotations import givens_decomposition

from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

In [2]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:3, :]
print("The matrix Q:")
print(Q)
print()

# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print("The sequence of parallel givens rotations:")
for parallel_set in givens_rotations:
    print(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi) for i, j, theta, phi in parallel_set])
print()

# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print("The matrix VQ:")
print(W)
print()

# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

print("The matrix VQU^\dagger:")
print(V.dot(Q.dot(U.T.conj())))
print()

# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("VQU^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

The matrix Q:
[[-0.007-0.411j -0.033-0.364j -0.031+0.2j   -0.326-0.304j -0.399+0.329j -0.434+0.043j]
 [ 0.056-0.5j   -0.041+0.288j -0.084-0.066j -0.538+0.26j   0.113-0.508j -0.071-0.135j]
 [ 0.149+0.017j  0.569+0.247j -0.168+0.631j -0.106-0.029j  0.244+0.112j  0.022+0.286j]]

The sequence of parallel givens rotations:
['2, 3, 0.609, 0.070']
['1, 2, 0.797, -1.281', '3, 4, 0.723, -0.402']
['0, 1, 1.162, 2.999', '2, 3, 1.060, -1.763', '4, 5, 0.651, 3.102']
['1, 2, 1.110, -0.612', '3, 4, 1.431, 3.046']
['2, 3, 1.136, 2.015']

The matrix VQ:
[[ 0.263-0.298j  0.352-0.536j  0.347+0.411j -0.221-0.303j  0.000+0.j     0.000+0.j   ]
 [ 0.339+0.228j -0.425-0.047j -0.068-0.162j -0.062-0.591j  0.298+0.422j -0.000-0.j   ]
 [-0.137-0.318j -0.080+0.052j -0.386+0.094j -0.237-0.133j -0.497+0.312j -0.542+0.054j]]

Number of rotations to perform in parallel: 1
[[ 0.263-0.298j  0.352-0.536j  0.423+0.502j  0.000+0.j     0.000+0.j     0.000+0.j   ]
 [ 0.339+0.228j -0.425-0.047j  0.004+0.202j -0.124-0.572j  0.